# Classification

## Implementation

### Pipeline

The process is divided intro 3 independent steps using a scikit `Pipeline`. Its input is a list of pictures, with the keypoints and the descriptors. The steps are:

- SpatialPyramid: performs the clustering of the descriptors and calculates the histograms.
- StandardScaler: standarizes each column of the feature matrix.
- SVC: performs the classification.

Internally, the pipeline uses a cache in order to speed up repeated operations.

### GridSearchCV

We use scikit class `GridSearchCV` in order to compute the parameter combinations. Internally, it performs cross-validation with the different combinations and returns the best result or, in our case, a table with all the results. This table is the put inside a pandas `DataFrame` and returned.

We are using 5 as the cv parameter and creating as many jobs as ores in the computer.

### Histogram intersection kernel

We use the following implementation for the histogram intersection kernel:

```python
def histogram_intersection_kernel(x, u):
    n_samples, n_features = x.shape
    K = np.zeros((x.shape[0], u.shape[0]), dtype=np.float32)
    for d in range(n_samples):
        K[d, :] = np.sum(np.minimum(x[d], u), axis=1)
    return K
```

In [1]:
from argparse import Namespace

import numpy as np
from matplotlib import pyplot as plt

from descriptors.histogram_intersection_kernel import histogram_intersection_kernel
from main import main


def run_experiment(param_grid: dict):
    args = Namespace(train_path='../data/MIT_split/train',
                     test_path='../data/MIT_split/test',
                     cache_path='../.cache')
    return main(args, param_grid)


Test different kernels and penalty parameter of the error term "C" for the classifier.

In [ ]:
param_grid = {
    'classifier__kernel': ['linear', 'rbf', 'sigmoid', histogram_intersection_kernel],
    'classifier__C': np.logspace(-4, 4, 9),
}

results = run_experiment(param_grid)
results.loc[results.param_classifier__kernel == histogram_intersection_kernel, 'param_classifier__kernel'] = \
    "histogram_intersection"

results.pivot(index='param_classifier__C', columns='param_classifier__kernel', values='mean_test_score') \
    .plot.line(logx=True)

plt.xlabel('C')
plt.ylabel('accuracy')
plt.show()

Comment results above

Test different kernels and penalty parameter of the error term "C" for the classifier. Pyramid level is set to 1

In [ ]:
param_grid = {
    'classifier__kernel': ['linear', 'rbf', 'sigmoid', histogram_intersection_kernel],
    'classifier__C': np.logspace(-4, 4, 9),
    'transformer__levels': [1]
}

results = run_experiment(param_grid)
results.loc[results.param_classifier__kernel == histogram_intersection_kernel, 'param_classifier__kernel'] = \
    "histogram_intersection"

results.pivot(index='param_classifier__C', columns='param_classifier__kernel', values='mean_test_score') \
    .plot.line(logx=True)

plt.xlabel('C')
plt.ylabel('accuracy')
plt.show()

Comment results above. RBF should perform much better.

Test different kernels and kernel coefficients for the classifier.

In [ ]:
param_grid = {
    'classifier__kernel': ['linear', 'rbf', 'sigmoid', histogram_intersection_kernel],
    'classifier__gamma': np.logspace(-4, 4, 9)
}

results = run_experiment(param_grid)
results.loc[results.param_classifier__kernel == histogram_intersection_kernel, 'param_classifier__kernel'] = \
    "histogram_intersection"

results.pivot(index='param_classifier__gamma', columns='param_classifier__kernel', values='mean_test_score') \
    .plot.line(logx=True)

plt.xlabel('gamma')
plt.ylabel('accuracy')
plt.legend(loc='best')

plt.show()

Comment results above

Test the classifier using different amounts of samples randomly selected form the descriptors, which were obtained using dense_sift with different scale factors.

In [ ]:
param_grid = {
    'transformer__samples': np.linspace(10000, 100000, 5),
}

results = run_experiment(param_grid)

results.plot.line(x='param_transformer__samples', y='mean_test_score')

plt.plot(results.param_transformer__samples, results.mean_test_score)

plt.xlabel('samples')
plt.ylabel('accuracy')
plt.show()

Comment results above

Test different normalization for the descriptors.

In [ ]:
param_grid = {
    'transformer__norm': ['l1', 'l2', 'power'],
}

results = run_experiment(param_grid)

# Colormap needed until a bug is fixed in next version of pandas.
results.plot.bar(x='param_transformer__norm', y='mean_test_score', colormap='jet')

plt.xlabel('norm')
plt.ylabel('accuracy')
plt.show()

Comment results above

Test different number of clusters, multiples of 2, for the min_batch k-means.

In [ ]:
param_grid = {
    'transformer__n_cluster': np.logspace(8, 11, 8, base=2),
}

results = run_experiment(param_grid)

results.plot.line(x='param_transformer__n_cluster', y='mean_test_score')

plt.xlabel('n_cluster')
plt.ylabel('accuracy')
plt.show()

Comment results above